# Biostrings and ShortRead

## Setup

In [ ]:
library(ShortRead)
library(Biostrings)
library(tidyverse) # for %>%
library(gdata) # for humanReadable
set.seed(100)

In [ ]:
### Specify FASTQ file
myfqdir <- "/data/hts_2019_data/hts2019_pilot_rawdata/"
myfqname <- "4_2018_P_H1_S27_L001_R1_001.fastq.gz"

myfqfile <- file.path(myfqdir, myfqname)
tools::md5sum(myfqfile)

# Set up ouput directory
outdir = path.expand("~/work/scratch/shortreadtutorial")
dir.create(outdir)

## FASTQ Basics

In [ ]:
### Read entire FASTQ file into memory
myfq <- readFastq(myfqfile)

### Check class
class(myfq)

In [ ]:
### Get Summary for FASTQ file
myfq

In [ ]:
### Get number of reads
length(myfq)

In [ ]:
### Subset based on first three records
myfq[1:3]

In [ ]:
### Subset based on three randomly selected records
myfq[sample(seq_len(length(myfq)),3, replace=FALSE)]

In [ ]:
### Get sequences, phred scores and read ids for reads 1,2, 3 and 1 (add duplicate on
### purpose
myrecs <- myfq[c(1,2,3,1)]

ShortRead::sread(myrecs) -> myseqs

Biostrings::quality(myrecs) -> myphreds

ShortRead::id(myrecs) -> myids

### Compare classes
class(myfq)
class(myseqs)
class(myphreds)

### Get read ids

In [ ]:
myids

### Get sequencing reads

In [ ]:
myseqs

### Get phred scores

In [ ]:
myphreds

### Explore classes

In [ ]:
###? DNAStringSet

### Get slot names for objects

In [ ]:
slotNames(myseqs)

### bash command to read the first 12 lines from the fastqfile

In [ ]:
mycmd <- paste("zcat ", myfqfile, "| head -n 12")
mycmd

In [ ]:
system(mycmd, intern = TRUE)

### Read Quality Scores

In [ ]:
### Check encoding of phred scores
myphreds %>% encoding

In [ ]:
as(quality(myrecs), "matrix")


### Read Sequences

In [ ]:
### Get the sequences as character strings
as.character(myseqs)

In [ ]:
### Check for duplicate reads
myseqs %>% duplicated

In [ ]:
### Find Unique reads
myseqs %>% unique

In [ ]:
### Get rid of duplicate reads
myseqs %>% unique -> myseqs

In [ ]:
### letter frequency counting
Biostrings::alphabetFrequency(myseqs)
Biostrings::letterFrequency(myseqs, c("A","T"))

### Sequence Transformations

In [ ]:
### Reverse reads
myseqs %>% reverse

In [ ]:
### complement reads
myseqs %>% complement

In [ ]:
### Reverse complement reads
myseqs %>% reverseComplement

In [ ]:
### Translate to AA
myseqs %>% translate

In [ ]:
myseqs

### Subset the sequences

In [ ]:
myseqs

In [ ]:
Biostrings::subseq(myseqs, start=1, width=9)

In [ ]:
### Translate to AA
myseqs %>% 
    subseq(start=7) %>%
    translate

In [ ]:
### Translate to AA
myseqs %>% 
    subseq(start=7, end=75) %>%
    translate

## Searching

In [ ]:
as.character(myseqs)

In [ ]:
mypattern0 <- "CATGA"
matchPattern(pattern = mypattern0, myseqs[[1]], max.mismatch=0)

In [ ]:
# change a base to introduce a mismatch
mypattern1 <- "CAGGA"
matchPattern(pattern = mypattern1, myseqs[[1]], max.mismatch=0)

In [ ]:
# search allowing a single mismatch 
matchPattern(pattern = mypattern1, myseqs[[1]], max.mismatch=1)

### Search multiple sequences
use `vmatchPattern` to search multiple sequences

In [ ]:
vmatchPattern(pattern = mypattern0, myseqs, max.mismatch=0)

if we allow mismatches in our pattern we find hits in other sequences

In [ ]:
vmatchPattern(pattern = mypattern0, myseqs, max.mismatch=2)

### Count sequence matches

In [ ]:
vcountPattern(pattern = mypattern0, myseqs, max.mismatch = 0)

In [ ]:
vcountPattern(pattern = mypattern0, myseqs, max.mismatch = 1)

In [ ]:
vcountPattern(pattern = mypattern0, myseqs, max.mismatch = 2)

### Pairwise Alignment

In [ ]:
pairwiseAlignment(pattern = mypattern0, myseqs[1])

In [ ]:
pairwiseAlignment(pattern="ACACGTCTGAACTCCAGTCA", myseqs[1])

In [ ]:
pairwiseAlignment(pattern="ACACGTCTGAACAGTCA", myseqs[2])

## Streaming and Sampling

In [ ]:
object.size(myfq) %>% humanReadable

In [ ]:
file.size(myfqfile) %>% humanReadable

### Streaming
Stream a FASTQ rather than loading the whole thing into memory.

After initializing the streamer, each `yield` call will return the next `n=5` reads from the FASTQ

In [ ]:
mystream <- FastqStreamer(myfqfile, n = 5)
mystream

In [ ]:
object.size(mystream) %>% humanReadable

In [ ]:
### The first yield will retrieve records 1 through 5
cur_reads <- yield(mystream)
cur_reads

In [ ]:
object.size(cur_reads) %>% humanReadable

In [ ]:
ShortRead::id(cur_reads)

In [ ]:
### Check status of stream (this yield added 5 for a total of 0+5 = 5 records)
mystream

In [ ]:
### The second yield will retrieve records 6 through 10
cur_reads <- yield(mystream)
ShortRead::id(cur_reads)

In [ ]:
### Check status of stream (this yield added 5 for a total of 5+5 = 10 records)
mystream

In [ ]:
### Be sure to close the stream
close(mystream)

In [ ]:
### Check status of stream
mystream

#### Double Check

In [ ]:
mystream <- FastqStreamer(myfqfile, 5)
cur_reads <- yield(mystream)
ShortRead::id(cur_reads)
close(mystream)

In [ ]:
mystream <- FastqStreamer(myfqfile, 5)
cur_reads <- yield(mystream)
ShortRead::id(cur_reads)
close(mystream)

### Sampling
Random sampling without reading in entire file upfront.

After initializing the sampler, each `yield` call returns `n=5` random reads from the FASTQ

In [ ]:
mysampler <- FastqSampler(myfqfile, n=5)

sample_reads <- yield(mysampler)
ShortRead::id(sample_reads)

close(mysampler)

In [ ]:
mysampler <- FastqSampler(myfqfile, n=5)

sample_reads = yield(mysampler)
ShortRead::id(sample_reads)

close(mysampler)

In [ ]:
### Reproducible random sampling
set.seed(18381)
mysampler <- FastqSampler(myfqfile, n=5)

sample_reads = yield(mysampler)
ShortRead::id(sample_reads)

close(mysampler)

In [ ]:
### Reproducible random sampling
set.seed(18381)
mysampler <- FastqSampler(myfqfile, n=5)

sample_reads = yield(mysampler)
ShortRead::id(sample_reads)

close(mysampler)

In [ ]:
mysampler <- FastqSampler(myfqfile, n=5)
sample_reads = yield(mysampler)
object.size(mysampler) %>% humanReadable
object.size(sample_reads)%>% humanReadable
close(mysampler)

### Iterate over fastq file in chunks of 5000 reads

In [ ]:
fq <- FastqStreamer(myfqfile, 5000)

while (length(reads <- yield(fq))) {
    print(reads)
}

close(fq)

## QC Report

### Get 2019 fastq filenames for the four lanes of sample 3 (2019 libraries)

- /data/hts_2019_data/hts2019_pilot_rawdata//1_2019_P_M1_S1_L001_R1_001.fastq.gz
- /data/hts_2019_data/hts2019_pilot_rawdata//1_2019_P_M1_S1_L002_R1_001.fastq.gz
- /data/hts_2019_data/hts2019_pilot_rawdata//1_2019_P_M1_S1_L003_R1_001.fastq.gz 
- /data/hts_2019_data/hts2019_pilot_rawdata//1_2019_P_M1_S1_L004_R1_001.fastq.gz

In [ ]:
### See https://krijnhoetmer.nl/stuff/regex/cheat-sheet/
### This pulls 2018 samples
myregex <- "^3_"
fqfiles <- list.files(myfqdir, pattern = myregex, full.names = TRUE) 
fqfiles

In [ ]:
myregex <- "^3_2019"
fqfiles <- list.files(myfqdir, pattern = myregex, full.names = TRUE) 
fqfiles

In [ ]:
### Create QA report for the four lanes (serial execution)
fqfiles %>% 
    ShortRead::qa(type = "fastq", BPPARAM = SerialParam()) %>% 
        report(dest=file.path(outdir, "2019tutorial-serial"))

### Create QA report for the four lanes (parallel execution).
```
# Does not work! Problem with BIOCParallel?
fqfiles %>% 
    ShortRead::qa(type = "fastq", BPPARAM = MulticoreParam(workers = 4)) %>% 
        report(dest=file.path(outdir, "2019tutorial-par"))
```

## The End

In [ ]:
sessionInfo()
q(save = "no")